## Demo: Sistema RAG

In [1]:
# ChromaDB
import chromadb
client = chromadb.Client()
client = chromadb.PersistentClient(path=".")

# Pandas
import pandas as pd

#Transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
from sentence_transformers import SentenceTransformer
#Otros
import openai
import os
from PyPDF2 import PdfReader

#Langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
import uuid  # Para generar un ID único para cada fragmento

c:\Users\nicoa\miniconda3\envs\sitiorama_api\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
api_key = "xd"

## Leer la carpeta de PDF e indexar su contenido en Base de datos vectoriales

In [3]:
# Ruta a la carpeta que contiene los archivos PDF
folder_path = 'PDF_pruebas'

# Obtener la lista de archivos en la carpeta
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

# Crear un "splitter" con un tamaño de chunk más pequeño
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Inicializar la base de datos ChromaDB
embeddings = OpenAIEmbeddings(openai_api_key=api_key)  # Asumiendo que estás usando OpenAI para obtener embeddings

# Nombre de la colección en ChromaDB
collection_name = "pruebas_sitiorama"

# Leer el contenido de cada archivo PDF y indexarlo
all_texts = []  # Lista para almacenar todos los fragmentos de texto
all_metadatas = []  # Lista para almacenar los metadatos (archivo origen)
all_ids = []  # Lista para almacenar los IDs

C:\Users\nicoa\AppData\Local\Temp\ipykernel_14208\1481984706.py:11: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)  # Asumiendo que estás usando OpenAI para obtener embeddings


In [4]:
import re

for pdf_file in pdf_files:
    pdf_path = os.path.join(folder_path, pdf_file)
    with open(pdf_path, 'rb') as file:
        reader = PdfReader(file)
        text = ""
        # Extraer el texto de todas las páginas
        for page in reader.pages:
            text += page.extract_text()

        # Eliminar saltos de línea innecesarios
        text = re.sub(r'\n+', ' ', text)  # Sustituye saltos de línea por un espacio
        # Eliminar espacios adicionales
        text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno solo
        # Eliminar espacios antes de los signos de puntuación (si es necesario)
        text = re.sub(r'\s([?.!,¿])', r'\1', text)
        # Opcional: eliminar espacios al inicio o final
        text = text.strip()        
        # Dividir el texto en fragmentos manejables
        chunks = splitter.split_text(text)

        # Crear un ID único para cada fragmento
        ids = [str(uuid.uuid4()) for _ in range(len(chunks))]
        
        # Agregar los fragmentos, metadatos e IDs a las listas
        all_texts.extend(chunks)
        all_metadatas.extend([{"source": pdf_file}] * len(chunks))
        all_ids.extend(ids)

# Inicializar Chroma con los documentos
db = Chroma.from_texts(all_texts, embeddings, metadatas=all_metadatas, ids=all_ids, collection_name=collection_name)

## Preguntas a Chroma DB

In [8]:
# Ahora db contiene los documentos indexados de los PDFs, puedes hacer consultas sobre ellos
# Ejemplo de consulta:
query = "Si quiero ir para miraflores que micro debo tomar?"
results = db.similarity_search(query, k=10)  # Limita los resultados a 10 documentos más similares

# Mostrar los resultados
print(f"Resultados para la consulta '{query}':")
for result in results:
    print(result)

Resultados para la consulta 'Si quiero ir para miraflores que micro debo tomar?':
page_content='urbano de forma exitosa. Con el fin de apoyar el proyecto de transporte público fluvial y consolidar a este modo se está implementando una red de muelles o puntos de embarque. Con fondos de la Subsecretaría de Desarrollo Regional y Administrativo se es - tán construyendo seis muelles públicos, ubica - dos en La Peña, Los Castaños, Carampangue, Collico, terminal de buses en el recinto de vialidad bajo el puente Calle-Calle. Estas obras están destinadas a apoyar la incorporación de “taxis fluviales” con tecnología sostenible como un nuevo modo de transporte público. El costo de inversión es de $249.000.000, financiados por PMU y MTT.cas con la concesión realizada en Santiago, a 15 años plazo, denominada Estaciones de Trans - bordo para Transantiago, que consiste básica - mente en que se selecciona una concesionaria privada para la construcción, mantenimiento y explotación de andenes y refugios

## RAG con ChatGPT

In [9]:
# Concatenar los resultados obtenidos por ChromaDB en un formato apropiado
context = "\n".join([result.page_content for result in results])

# Configuramos el cliente de OpenAI con la clave API
chatgpt = openai.OpenAI(api_key=api_key)
    
# Crear el prompt para GPT-4
prompt = f"Eres un recomendador de lugares de la ciudad de valdivia, chile , Genera una respuesta a la pregunta siguiente : {query}:\n\n"
prompt += f"Datos de contexto: {context}"

# Hacemos la consulta a ChatGPT
chat_completion = chatgpt.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,  # Aquí enviamos el prompt modificado
            }
        ],
        model="gpt-4o-mini",  # Modelo que quieres usar
    )
    
# Obtener la reseña generada por GPT-4
respuesta = chat_completion.choices[0].message.content
print(respuesta)

Para llegar a Miraflores en Valdivia, tienes varias opciones de transporte público que puedes considerar. La micro que te conviene tomar es la que tiene la ruta que conecta con esa zona específica. Generalmente, las micros que circulan por la ciudad tienen sus paradas claramente señalizadas, así que te recomiendo que busques la información en el paradero más cercano o preguntes a los conductores sobre la ruta.

Además, ten en cuenta que el sistema de transporte público está en desarrollo y se están implementando nuevas conexiones y muelles para enriquecer la movilidad en la ciudad. Así que siempre es una buena idea consultar con los usuarios locales o revisar aplicaciones de transporte que puedan ofrecer información actualizada sobre las rutas disponibles y los horarios.

Recuerda también que puedes disfrutar de las bellezas del paisaje valdiviano durante el trayecto, ya que la ciudad está rodeada de naturaleza y ríos que hacen de tu viaje una experiencia más placentera. ¡Buen viaje!


## Refactorización para tener una función

In [9]:
# Función para obtener respuesta de GPT-4 basada en los documentos indexados
def obtener_respuesta(query, db, api_key, k=10):
    # Obtener los resultados más relevantes desde la base de datos ChromaDB
    results = db.similarity_search(query, k=k)  # Limita los resultados a 'k' documentos más similares

    # Concatenar los resultados obtenidos por ChromaDB en un formato apropiado
    context = "\n".join([result.page_content for result in results])

    # Configuramos el cliente de OpenAI con la clave API
    chatgpt = openai.OpenAI(api_key=api_key)


    # Crear el prompt para GPT-4
    prompt = f"Genera una respuesta a la pregunta siguiente: {query}:\n\n"
    prompt += f"Datos de contexto: {context}"
    
    # Hacemos la consulta a ChatGPT
    chat_completion = chatgpt.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,  # Aquí enviamos el prompt modificado
            }
        ],
        model="gpt-4o-mini",  # Modelo que quieres usar
    )
    # Obtener la reseña generada por GPT-4
    respuesta = chat_completion.choices[0].message.content
    return respuesta

In [15]:
# Llamar a la función con una nueva pregunta
query = "¿Cuándo es el último día para ingresar las calificaciones al segundo semestre?"
respuesta = obtener_respuesta(query, db, api_key)

# Mostrar la respuesta
print(respuesta)

El último día para ingresar las calificaciones del segundo semestre 2024 es el **30 de diciembre de 2024**. Es importante que todas las unidades académicas realicen este ingreso a tiempo para asegurar una buena gestión académica.


In [11]:
# Llamar a la función con una nueva pregunta
query = "¿Cuándo es el monto de las remuneraciones de los académicos en 2024? Respuesta corta"
respuesta = obtener_respuesta(query, db, api_key)

# Mostrar la respuesta
print(respuesta)

El monto total de las remuneraciones de los académicos en 2024 es de M$ 49.191.161. Esto incluye M$ 39.078.240 en remuneraciones a académicos y M$ 2.527.918 en honorarios académicos.


In [12]:
# Llamar a la función con una nueva pregunta
query = "¿Cuáles son las obligaciones de los estudiantes? Respuesta corta"
respuesta = obtener_respuesta(query, db, api_key)

# Mostrar la respuesta
print(respuesta)

Las obligaciones de los estudiantes de la Corporación incluyen: respetar las normas éticas, legales y reglamentarias de la Universidad; mantener conductas de respeto y tolerancia en el ámbito académico; contribuir al desarrollo y prestigio de la institución; preservar su patrimonio moral y material; y esforzarse en su formación y desarrollo personal, científico y cultural. Además, deben actuar con honestidad en todas sus actividades académicas y pueden ser sancionados por faltas graves, menos graves o leves en caso de incumplimiento.
